<a href="https://colab.research.google.com/github/jperezaracil/03MAIR-Algoritmos-de-Optimizacion-2021/blob/main/Jorge_P%C3%A9rez_Aracil_Algoritmos(ABR21)_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3- Actividad Guiada 3<br>
Nombre: Jorge Pérez Aracil<br>
https://github.com/jperezaracil/03MAIR-Algoritmos-de-Optimizacion-2021

https://colab.research.google.com/drive/1rZsDDh8lstq0NOY_ozUKpaWRCxRCTLIP#scrollTo=i0SeS3aTT86U


#Carga de librerias

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

#Carga de los datos del problema

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)




('swiss42.tsp', <http.client.HTTPMessage at 0x7f6e6adb7b50>)

In [ ]:
#Modulos extras, no esenciales
import numpy as np
import matplotlib.pyplot as plt
import imageio                    #Para construir las imagenes con gif
from google.colab import files    #Para descargar ficheros generados con google colab

from tempfile import mkstemp      #Para genera carpetas y ficheros temporales

import random                     #Para generar valores aleatorios


In [ ]:
#Carga de datos y generación de objeto problem
# Descargamos el fichero con la matriz de distancias (hay varias opciones).
###############################################################################

# 42 Staedte Schweiz (Fricker)
file = "swiss42.tsp"
url = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp"

# 51-city problem (Christofides/Eilon)
#file = "eil51.tsp"
#url = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp"

# 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp"
#url = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp"

urllib.request.urlretrieve(url, file)

# Creamos el objeto que contendrá los datos.
problem = tsplib95.load(file)

# Nodos del problema.
nodos = list(problem.get_nodes())

# Matriz de distancias del problema.
matriz_distancias = [[problem.get_weight(i, j) for i in nodos] for j in nodos]  


In [ ]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
                
#dir(problem)

15

#Funcionas basicas


In [ ]:
# Genera una solución aleatoria que comienza en el nodo 0.
def crear_solucion(nodos):

  lista_nodos = nodos.copy()
  siguiente_nodo = lista_nodos.pop(0)
  solucion = [siguiente_nodo]
  

  while len(lista_nodos) > 0:
    siguiente_nodo = lista_nodos.pop(random.choice(range(len(lista_nodos))))
    solucion += [siguiente_nodo]
  
  return solucion


def distancia_total(solucion, matriz_distancias):
  distancia = 0
  
  for i in range(len(solucion)-1):
    distancia += matriz_distancias[solucion[i]][solucion[i+1]]
  

  distancia += matriz_distancias[solucion[len(solucion)-1]][solucion[0]]
  
  return distancia

 

#Busqueda Aleatoria

In [ ]:
def busqueda_aleatoria(matriz_distancias, iteraciones):
  mejor_distancia = float('inf')
  
  for i in range(iteraciones):
    solucion = crear_solucion(list(range(len(matriz_distancias))))
    distancia = distancia_total(solucion, matriz_distancias)
    
    #Actualizamos la solución
    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
  
  print("Mejor solución:", mejor_solucion)
  print("Distancia     :", mejor_distancia)
  
  return mejor_solucion
 

In [ ]:
# Búsqueda aleatoria con 5000 iteraciones.
solucion = busqueda_aleatoria(matriz_distancias, 5000)

Mejor solución: [0, 7, 15, 5, 9, 2, 22, 30, 25, 11, 18, 36, 13, 4, 27, 6, 8, 23, 40, 21, 24, 39, 41, 38, 31, 14, 29, 20, 37, 26, 16, 33, 3, 12, 34, 32, 10, 28, 17, 35, 19, 1]
Distancia     : 3662


#Busqueda Local  2-opt

In [ ]:

# Generador de soluciones vecinas 2-opt (intercambia 2 nodos).
# Se obtiene la mejor de todas ellas.
def genera_vecina(solucion, matriz_distancias):

  mejor_distancia = float('inf')

  for i in range(1, len(solucion) - 1):
    for j in range(i + 1, len(solucion)):
      # Se genera una nueva solución intercambiando los dos nodos 'i' y 'j':
      vecina = solucion.copy()
      vecina[i], vecina[j] = vecina[j], vecina[i]


      distancia_vecina = distancia_total(vecina, matriz_distancias)

      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
  return mejor_solucion
 

In [ ]:
# Función de búsqueda local
###############################################################################
# - Sobre el operador de vecindad 2-opt (función genera_vecina)
# - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(matriz_distancias):
  mejor_solucion = []
  
  # Se generar una solución inicial de referencia (aleatoria).
  solucion_referencia = crear_solucion(list(range(len(matriz_distancias))))
  mejor_distancia = distancia_total(solucion_referencia, matriz_distancias)

  # Se crea un contador para las iteraciones del algoritmo.
  iteracion = 0
  
  while(1):
    # Se incrementa el contador.
    iteracion += 1

    vecina = genera_vecina(solucion_referencia, matriz_distancias)
    distancia_vecina = distancia_total(vecina, matriz_distancias)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina

      solucion_referencia = vecina
    else:
      print("La mejor solución encontrada es:", mejor_solucion)
      print("Se ha encontrado en la iteracion", iteracion)
      print("Y tiene distancia:", mejor_distancia)
      
      return mejor_solucion


In [ ]:
solucion = busqueda_local(matriz_distancias)

La mejor solución encontrada es: [0, 37, 15, 16, 14, 19, 13, 10, 21, 40, 24, 38, 34, 33, 20, 6, 5, 26, 18, 4, 3, 1, 7, 17, 36, 35, 31, 12, 11, 25, 41, 23, 39, 22, 9, 8, 29, 30, 32, 28, 2, 27]
Se ha encontrado en la iteracion 31
Y tiene distancia: 1843


#Simulated Annealing

In [ ]:
# Funciones auxiliares
###############################################################################

# Generador de una única solucion vecina 2-opt aleatoria.
def genera_vecina_aleatoria(solucion):
  # Se eligen dos nodos aleatoriamente.
  i, j = sorted(random.sample(range(1, len(solucion)), 2))
  
  # Se obtiene la solución vecina resultante de intercambiar dichos nodos.
  vecina = solucion.copy()
  vecina[i], vecina[j] = vecina[j], vecina[i]

  return vecina

# Función de probabilidad para aceptar peores soluciones.
def probabilidad(T, d):
  return random.random() < math.exp(-d / T)

# Función de descenso exponencial de temperatura.
def bajar_temperatura(T):
  return 0.99 * T


In [ ]:
# Función principal
###############################################################################
def recocido_simulado(matriz_distancias, temperatura):
  # Se generar una solución inicial de referencia (aleatoria).
  solucion_actual = crear_solucion(list(range(len(matriz_distancias))))
  distancia_actual = distancia_total(solucion_actual, matriz_distancias)
  
  mejor_distancia = float('inf')
  
  while temperatura > 0.0001:
    vecina = genera_vecina_aleatoria(solucion_actual)
    distancia_vecina = distancia_total(vecina, matriz_distancias)
    
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    

    if distancia_vecina < distancia_actual or \
    probabilidad(temperatura, distancia_vecina - distancia_actual):
      solucion_actual = vecina
      distancia_actual = distancia_vecina
    
    temperatura = bajar_temperatura(temperatura)
  
  print("La mejor solución encontrada es:", mejor_solucion)
  print("Y tiene distancia:", mejor_distancia)
  
  return mejor_solucion



In [ ]:
solucion  = recocido_simulado(matriz_distancias, 10000000)
 



La mejor solución encontrada es: [0, 37, 36, 35, 20, 33, 34, 32, 31, 17, 7, 5, 4, 26, 13, 16, 15, 14, 19, 18, 41, 23, 21, 2, 27, 28, 29, 9, 8, 10, 12, 11, 25, 40, 24, 39, 22, 38, 30, 3, 6, 1]
Y tiene distancia: 1865


## Algoritmo genético

In [ ]:
# Funciones auxiliares
# NOTA: Son una posible opción, pero todas se pueden modificar.
###############################################################################
import copy
# Generación de una población inicial de soluciones de tamaño 'n'.
def generar_poblacion(nodos, n):
  return [crear_solucion(nodos) for _ in range(n)]

def evaluar_poblacion(poblacion, matriz_distancias):
  mejor_distancia = float('inf')
  
  for solucion in poblacion:
    distancia = distancia_total(solucion, matriz_distancias)
    
    # Si la solución es mejor que la que teníamos, se actualiza.
    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
  
  return mejor_solucion, mejor_distancia


def cruzar(poblacion, nodos, mutacion):

  lista_padres = copy.deepcopy(poblacion)
  poblacion_ampliada = copy.deepcopy(poblacion)


  while len(lista_padres) > 1:
    padre_1, padre_2 = random.sample(lista_padres, 2)
    lista_padres.remove(padre_1)
    lista_padres.remove(padre_2)
    
    poblacion_ampliada.extend(descendencia([padre_1, padre_2], nodos, mutacion))
  
  return poblacion_ampliada


def descendencia(padres, nodos, mutacion):
  # Se obtiene un punto de corte aleatorio.
  corte = random.sample(range(1, len(padres[0])), 1)[0] #
  

  hijo_1 = factibilizar(padres[0][:corte] + padres[1][corte:], nodos)
  hijo_2 = factibilizar(padres[1][:corte] + padres[0][corte:], nodos)
  
  # Se devuelven las soluciones hijas con y sin mutación.
  return [hijo_1, hijo_2, mutar(hijo_1, mutacion), mutar(hijo_2, mutacion)]


def factibilizar(solucion, nodos):
  # Se obtiene una lista con los nodos que no aparecen en la solución.
  nodos_faltantes = [item for item in nodos if item not in solucion]


  for i in range(len(solucion)):
    if solucion[i] in solucion[:i]:
      solucion[i] = nodos_faltantes.pop(0)
  
  return solucion


def mutar(solucion, mutacion):
  if random.random() < mutacion:
    # Se obtienen dos nodos aleatorios que no sean el nodo inicial y se
    # intercambian.
    i, j = random.sample(solucion[1:], 2)
    solucion[i], solucion[j] = solucion[j], solucion[i]
  
  return solucion

def seleccionar(poblacion, matriz_distancias, n, elitismo):

  poblacion_desordenada = [[distancia_total(solucion, matriz_distancias), \
                            solucion] for solucion in poblacion]
  poblacion_ordenada = sorted(poblacion_desordenada, key = lambda x: x[0])

  elite = [x[1] for x in poblacion_ordenada][:int(n * elitismo)]
  no_elite = [x[1] for x in poblacion_ordenada][int(n * elitismo):]
  repesca = random.sample(no_elite, int(n * (1 - elitismo)))

  return elite + repesca

In [ ]:
# Función principal del algoritmo genético
###############################################################################
def algoritmo_genetico(matriz_distancias, n = 100, generaciones = 100, \
                       mutacion = 0.15, elitismo = 0.1):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  # Se genera la población inicial.
  poblacion = generar_poblacion(list(range(len(matriz_distancias))), n)
  
  # Obtenemos la mejor solución inicial.
  mejor_solucion, mejor_distancia = evaluar_poblacion(poblacion, \
                                                      matriz_distancias)
  
  # Se inicia el ciclo de generaciones realizando tantas como indique la
  # variable "generaciones".
  for iteracion in range(generaciones):
    # Se realiza el cruce de la población (incluyendo la mutación).
    poblacion = cruzar(poblacion, list(range(len(matriz_distancias))), mutacion)

    # Se selecciona a la nueva población para la siguiente generación.
    poblacion = seleccionar(poblacion, matriz_distancias, n, elitismo)
    
    # Se evalúa la nueva población para hallar la mejor solución actual.
    mejor_solucion, mejor_distancia = evaluar_poblacion(poblacion, \
                                                        matriz_distancias)
    
    print(f"\nGeneración #{iteracion}:")
    print(f"La mejor solución es:{mejor_solucion}")
    print(f"Y tiene distancia: {mejor_distancia}")
  
  return mejor_solucion

In [ ]:
# Primer intento: 2211
#sol = algoritmo_genetico(matriz_distancias, n = 100, generaciones = 200, mutacion = 0.15, elitismo = 0.1)

# Segundo intento (aumentamos la poblacion): 1870
#sol = algoritmo_genetico(matriz_distancias, n = 500, generaciones = 200, mutacion = 0.15, elitismo = 0.1)

# Tercer intento (aumentamos las generaciones): 1751
#sol = algoritmo_genetico(matriz_distancias, n = 500, generaciones = 250, mutacion = 0.15, elitismo = 0.1)

# Cuarto intento (aumentamos el elitismo): 1632
#sol = algoritmo_genetico(matriz_distancias, n = 500, generaciones = 250, mutacion = 0.15, elitismo = 0.4)

# Quinto intento (aumentamos la mutación): 1487
sol = algoritmo_genetico(matriz_distancias, n = 500, generaciones = 250, mutacion = 0.3, elitismo = 0.4)
#sol = [0, 7, 17, 37, 15, 16, 14, 19, 13, 5, 26, 12, 18, 6, 1, 4, 3, 27, 2, 28, 30, 29, 9, 8, 10, 11, 25, 41, 23, 40, 24, 21, 39, 22, 38, 31, 36, 35, 20, 33, 34, 32]


Generación #0:
La mejor solución es:[0, 6, 38, 29, 28, 5, 7, 17, 31, 35, 19, 23, 24, 20, 34, 1, 2, 30, 33, 18, 3, 27, 26, 36, 37, 22, 21, 12, 40, 13, 15, 8, 9, 10, 14, 16, 11, 25, 39, 41, 4, 32]
Y tiene distancia: 3777

Generación #1:
La mejor solución es:[0, 10, 11, 18, 21, 32, 8, 14, 16, 2, 28, 9, 22, 39, 41, 3, 13, 7, 6, 26, 15, 40, 24, 20, 17, 19, 5, 27, 12, 25, 4, 30, 29, 1, 31, 23, 33, 34, 37, 35, 36, 38]
Y tiene distancia: 3744

Generación #2:
La mejor solución es:[0, 6, 38, 29, 28, 5, 7, 17, 31, 35, 19, 23, 24, 20, 34, 1, 2, 30, 33, 18, 3, 27, 26, 36, 37, 22, 21, 9, 8, 11, 16, 14, 4, 10, 12, 13, 15, 25, 32, 39, 40, 41]
Y tiene distancia: 3569

Generación #3:
La mejor solución es:[0, 1, 35, 22, 38, 20, 21, 24, 40, 32, 8, 11, 2, 6, 13, 16, 28, 27, 9, 5, 14, 36, 37, 15, 19, 18, 25, 12, 10, 23, 4, 3, 7, 17, 33, 34, 30, 31, 29, 39, 26, 41]
Y tiene distancia: 3386

Generación #4:
La mejor solución es:[0, 1, 35, 22, 38, 20, 21, 24, 40, 32, 8, 11, 2, 6, 13, 16, 28, 27, 9, 5, 14, 36, 3